# ERUPT under simulated random assignment

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now..

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# the below checks for whether we run dowhy, causaltune, and FLAML from source
root_path = root_path = os.path.realpath('../..')
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "causaltune"))

import pandas as pd
import numpy as np

from causaltune.erupt import DummyPropensity, ERUPT

## Creating synthetic data for the experiment with fully random assignment

In [2]:
# Let's create some synthetic data
n=10000

# Let's create a dataset with a single feature
df = pd.DataFrame({"X": np.random.uniform(size=n)})

# Now let's create a response-to-treatment function that correlates with the feature
def outcome(x: np.ndarray, treatment: np.ndarray) -> np.ndarray:
    return 2*np.random.uniform(size=len(x)) + x*(treatment == 1)

# Let's consider a fully random treatment
df["T1"] = np.random.randint(0, 2, size=n)
# and simulate the corresponing experiment outcomes
df["Y1"] = outcome(df["X"], df["T1"])
df.head()

,X,T1,Y1
0,0.528828,0,1.472417
1,0.712731,0,0.774130
2,0.796902,1,2.005292
3,0.788617,1,2.038432
4,0.130933,1,2.099868


## Creating synthetic data for the experiment with biased assignment

In [3]:
# Let's consider another experiment on the same population, but with 
# treatment assignment that's biased by the feature, because we believe that 
# customers with higher values of the feature will be more responsive to the treatment

df["p"] = 0.5+0.5*df["X"] # probability of binary treatment being applied
df["T2"] = (np.random.rand(len(df)) <df["p"]).astype(int) # sample with that propensity

# We really only need the ex ante probability of the treatment that actually was applied
# This will work exactly the same way in a multi-treatment case
df["p_of_actual"] = df["p"]*df["T2"] + (1-df["p"])*(1-df["T2"])

# Now let's evaluate the outcome for this experiment

df["Y2"] = outcome(df["X"], df["T2"])

df.head()

,X,T1,Y1,p,T2,p_of_actual,Y2
0,0.528828,0,1.472417,0.764414,1,0.764414,1.360950
1,0.712731,0,0.774130,0.856365,1,0.856365,2.104692
2,0.796902,1,2.005292,0.898451,1,0.898451,2.144687
3,0.788617,1,2.038432,0.894308,1,0.894308,1.983497
4,0.130933,1,2.099868,0.565467,1,0.565467,0.942264


## Create a simple function to calculate estimated outcomes for alternate assignments

In [4]:
# Let's define a simple function that takes the actual experiment data and the hypothetical policy,
# and returns the average outcome under the hypothetical policy
def evaluate_erupt(actual_propensity: pd.Series,
                   actual_treatment: pd.Series,
                   actual_outcome: pd.Series,
                   hypothetical_policy: pd.Series) -> float:
    
    # define a dummy propensity model that will simply return the propensities when calling predict_proba
    propensity_model = DummyPropensity(p=actual_propensity, treatment=actual_treatment)

    # obtain ERUPT value under hypothetical policy
    e = ERUPT(treatment_name="T", propensity_model=propensity_model)
    return e.score(df=pd.DataFrame({"T": np.array(actual_treatment)}), outcome=actual_outcome, policy=hypothetical_policy)

## Estimate random assignment outcome from biased assignment experiment

In [5]:
# Let's use data from biased assignment experiment to estimate the average effect of fully random assignment
est = evaluate_erupt(actual_propensity=df["p_of_actual"], 
                     actual_treatment=df["T2"],
                     actual_outcome=df["Y2"],
                     hypothetical_policy=df["T1"])


print("Average outcome of the actual biased assignment experiment:", df["Y2"].mean())
print("Estimated outcome of random assignment:", est)
print("Average outcome of the actual random assignment experiment:",  df["Y1"].mean())

Average outcome of the actual biased assignment experiment: 1.4186689330175457
Estimated outcome of random assignment: 1.2463633587748948
Average outcome of the actual random assignment experiment: 1.2462788233315247


## Estimate biased assignment outcome from random assignment experiment

In [6]:
# Conversely, we can take the outcome of the fully random test and use it to estimate what the outcome of the biased assignment would have been

# Let's use data from biased assignment experiment to estimate the average effect of fully random assignment
hypothetical_policy = df["T2"]
est = evaluate_erupt(actual_propensity=0.5*pd.Series(np.ones(len(df))), 
                     actual_treatment=df["T1"],
                     actual_outcome=df["Y1"],
                     hypothetical_policy= df["T2"])

print("Average outcome of the actual random assignment experiment:", df["Y1"].mean())
print("Estimated outcome of biased assignment:", est)
print("Average outcome of the actual biased assignment experiment:",  df["Y2"].mean())

Average outcome of the actual random assignment experiment: 1.2462788233315247
Estimated outcome of biased assignment: 1.4054069453155034
Average outcome of the actual biased assignment experiment: 1.4186689330175457


For more details on the math behind ERUPT, consult [Hitsch and Misra (2018)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3111957), who call it policy value. Note also that we assume that treatment takes integer values from 0 to n.